# SudokuPlantDesign.jl - Basic Example

This notebook serves as a basic usage guide to the module `SudokuPlantDesign.jl`. It is devided into the following general parts:
1. Importing modules
2. Generating an initial check distribution
3. Defining the optimization parameters
4. Performing the optimization

Each part comes with code and examples.

### 1) Installing and Importing modules

To utilize `SudokuPlantDesign` functions within the notebook (and also `PyPlot` functions, which come in handy later), use

In [ ]:
using SudokuPlantDesign
using PyPlot

### 2) Generate initial (random) check distribution

To start a new plant design, the first step is to generate a new configuration of checks, distributed over the available are. Start by generating such a new configuration `conf`. In this example, it consists of 2 horizontal and 5 vertical blocks of dimensions `10` x `2` respectively. In total, there are `4` different types of checks in the configuration.

As an optional argument, the boundary conditions (present in the optimization algorithm later on) can be passed as an argument `bc`. Values are either `:periodic` or `open`. 

In [ ]:
conf = get_configuration(
    [10,10], [2,2,2,2,2],  4
    ;
    bc = :open
);

To visualize the configuration `conf`, use `show_configuration`. If you are seeing a uniform color pattern, remember that initially all plots are assumed to be entries.

In [ ]:
show_configuration(conf)

You can designate empty plots which will be ignored in the sudoku optimization (missing plots, etc). In this example, leave the plot `[1,3]` and `[1,4]` empty. Configuration is plotted again to show the update.

In [ ]:
empty_plots!(conf, 1:1,3:4)
show_configuration(conf)

For moving on to the actual optimization, the different checks are initialized randomly with one check per check type per block.

In [ ]:
initialize_checks_per_block!(conf)
show_configuration(conf)

At this point, saving the resulting configuration might be a good idea. This can be done by utilizing the `PyPlot` command `savefig` which saves the current figure as an image file.

In [ ]:
show_configuration(conf)

mkpath("output/")
savefig("output/initial_random_check_distribution.png")

### 3) Define optimization parameters

In order to optimize the check distribution, one has to specify two aspects explicitly:

- cost functions / what are good or bad check distributions?
- updates / how do you generate a new check distribution from an old one?

#### Cost functions

The *cost function* defines a function that associates a value to every distribution of checks, i.e. it measures how well the checks are distributed. In `SudokuPlantDesign`, it is implemented as a julia function that takes the configuration `conf` as an input and returns a numeric value. Lower values represent lower costs and are therefore desirable. If you want to associated an interpretation to the actual cost value, you can think of it as *how far is the check distribution still away from being perfect*.

In practice, the overall cost function can be built from small, already implemented pieces in e.g. the following way:

In [ ]:
function K_indiv(conf :: C) :: Float64 where {C <: CheckConfiguration}
    return  K_checks_per_type_per_block(conf, 1)*20+
            K_neighbors_different_check_functional(conf, d->1/(d^3))*0.5 +
            K_neighbors_same_check_functional(conf, d->1/(d^3))
end

Here, the following partial cost functions are used:
- `K_checks_per_type_per_block` : Gives costs if there are checks which occur more than `1` time per check type and block. Optimizes to get an exact amount of checks per type per each block. Additionally, here it is weighted by a factor of `20`.
- `K_neighbors_different_check_functional` : Gives costs when checks of different types are neighbors, i.e. it acts as a repulsive potential of different neighboring checks (e.g. check 1 and check 3) with the functional dependence $\text{costs} \sim 1/\text{distance}^3$. Additionally, here it is weighted by a factor of `0.5`.
- `K_neighbors_same_check_functional` : Gives costs when checks of the same check type are neighbors, i.e. it acts as a repulsive potential of neighboring checks of equal type (e.g. check 1 and check 1) with the functional dependence $\text{costs} \sim 1/\text{distance}^3$. 

Summarizing, one could say that the cost function defines the *goal* of the optimization in terms of judging what is *good* and *bad*.

#### Updates.
Select which updates you want to perform during optimization. `UpdateNewCheckLabel` takes a random check in the configuration and assigns a new check label to it, `UpdateSwapCheckCheck` swaps checks with other checks, and `UpdateSwapCheckEntry` swaps checks with entries.

In [ ]:
updates = [
    UpdateNewCheckLabel(),
    UpdateSwapCheckCheck(),
    UpdateSwapCheckEntry()
]

### 4) Run sudoku optimization

To run the optimization algorithm (with parameters defined above), execute the following cell which will run 100000 consecutive updates. The function `optimize_design!` needs the following parameters:
- `conf` the configuration to optimize
- `updates` the list of updates to used
- `K_indiv` the cost function which defines the costs of a configuration
- number of consecutive update steps to run

the function returns and array containing the cost function values after each update step. This array can either be discarded or saved for later analysis.

In [ ]:
cost_values = optimize_design!(
    conf,
    updates,
    K_indiv,
    100000
);

To analyze the output of the Sudoku optimization, one has several options.


First and foremost, one can simply look at the resulting configuration

In [ ]:
show_configuration(conf)

Second, one can also print numerical details of the resulting configuration

In [ ]:
print_info(conf)

Third, one can analyze the cost function values which were returned during the optimization. The absolute values of costs are not as important as the relative improvement during the optimization. Therefore, for plotting it is advised to subtract the minimal cost value and plot on a logarithmic scale. From this plot, one can infer if the optimization converged or if further updates are needed.

In [ ]:
# print cost reduction
println("cost values have been reduced during the optimization:\n",cost_values[1], " -> ", cost_values[end])

# plot cost reduction (log scale)
figure()
plot(cost_values .- minimum(cost_values) .+1)
yscale("log")